In [1]:
import os 
os.chdir('../')

In [2]:
%pwd

'e:\\EndToEnd\\kidney-disease-classification'

## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    image_size: list
    learning_rate: float
    include_top: bool
    weights: str
    classes: int

## Config

In [9]:
import os
from kidneyDiseaseClassifire.utils.common import read_yaml, create_dirs
from kidneyDiseaseClassifire.constants import *

class ConfigManger:
    def __init__(self,
                 config = CONFIG_FILEPATH,
                 params = PARAMS_FILEPATH):
        
        self.config = read_yaml(config)
        self.params = read_yaml(params)
    
    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.base_model

        create_dirs([config.root_dir])

        return BaseModelConfig(
            root_dir= config.root_dir,
            base_model_path= config.base_model_path,
            updated_model_path= config.updated_model_path,
            image_size= self.params.IMAGE_SIZE,
            learning_rate= self.params.LEARNING_RATE,
            include_top= self.params.INCLUDE_TOP,
            classes= self.params.CLASSES,
            weights= self.params.WEIGHTS
        )

## Components

In [10]:
import os
import tensorflow as tf
import urllib.request as request
from zipfile import ZipFile


class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config
    
    def download_base_model(self):
        self.model = tf.keras.applications.MobileNetV2(
            input_shape=self.config.image_size,
            include_top=self.config.include_top,
            weights=self.config.weights
        )
        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    @staticmethod
    def update_base_model(
        model,
        classes,
        freeze_all,
        freeze_till,
        learning_rate
         ):

        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        outputs = tf.keras.layers.Dense(classes, activation='softmax')(flatten_in)

        full_model = tf.keras.Model(inputs=model.inputs, outputs=outputs)

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        return full_model
    
    def get_base_model(self):
        self.full_model = self.update_base_model(
            model=self.model,
            classes=self.config.classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.learning_rate
        )

        self.save_model(path=self.config.updated_model_path, model=self.full_model)

## pipeline

In [11]:
from kidneyDiseaseClassifire import logger
try:
    model_config = ConfigManger().get_base_model_config()
    base_model = BaseModel(config=model_config)
    logger.info("Downloading base model")
    base_model.download_base_model()
    logger.info("Base model downloaded successfully")
    base_model.get_base_model()
    logger.info("Updated model created successfully")
except Exception as e:
    raise e

[2024-01-23 23:48:40,744: INFO: common: Yaml file loaded successfully]
[2024-01-23 23:48:40,749: INFO: common: Yaml file loaded successfully]
[2024-01-23 23:48:40,751: INFO: common: Created directory at artifacts/base_model]
[2024-01-23 23:48:40,752: INFO: 3539781943: Downloading base model]
[2024-01-23 23:48:43,054: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-01-23 23:48:43,389: INFO: 3539781943: Base model downloaded successfully]
[2024-01-23 23:48:43,764: INFO: 3539781943: Updated model created successfully]
